# Analyse the limiting factors of contrail formation

_Dataset:_ Supplementary data for Megill and Grewe (2024): "Investigating the limiting aircraft design-dependent and environmental factors of persistent contrail formation".

_Authors:_

- Liam Megill (1, 2), https://orcid.org/0000-0002-4199-6962   
- Volker Grewe (1, 2), https://orcid.org/0000-0002-8012-6783  

_Affiliation (1)_: Deutsches Zentrum für Luft- und Raumfahrt (DLR), Institut für Physik der Atmosphäre, Oberpfaffenhofen, Germany

_Affiliation (2)_: Delft University of Technology (TU Delft), Faculty of Aerospace Engineering, Section Aircraft Noise and Climate Effects (ANCE), Delft, The Netherlands

_Corresponding author_: Liam Megill, liam.megill@dlr.de

_doi_: https://doi.org/10.5194/egusphere-2024-3398

---


### Summary
This notebook shows the vast majority of the limiting factors analyses in the linked paper. Specifically, the notebook creates Figures 3, 4 and 5, Supplementary Figures 4 and 5 and the data in Table 2 and Supplementary Table 1. We start by analysing the limiting factors of persistent contrail formation within each grid cell as well as the potential persistent contrail formation $p_{pcf}$ for all aircraft designs and RHi enhancements. We then analyse the boundaries of persistent contrail formation regions. 

### Inputs
- `data/processed/limfac/nonborder_limfac_allAC_rmS_ERA5_GRIB_allcorr.nc`: Non-boundary limiting factor results for all aircraft and RHi enhancements.
- `data/processed/limfac/vert_limfac_allAC_rmS_ERA5_GRIB_allcorr_v3.nc`: Vertical limiting factor boundary results for all aircraft and RHi enhancements.
- `data/processed/limfac/limfac_allAC_rmS_ERA5_GRIB_allcorr_v3.nc`: Horizontal limiting factor boundary results for all aircraft and RHi enhancements.
- `data/processed/limfac/areas_grib.pickle`: Grid cell areas
- `data/procssed/limfac/perimeters_grib.pickle`: Perimeters of grid cells
- `data/external/DEPA2050/FP_con_2050.txt`: DEPA 2050 progressive global aviation scenario in the year 2050. This can be downloaded from https://doi.org/10.5281/zenodo.11442323.

### Outputs
- Figures 3, 4 and 5
- Supplementary Figures 4 and 5
- Table 2
- Supplementary Table 1

---

### Copyright

Copyright © 2024 Liam Megill

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0. Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

Please run this cell before any others in this notebook! The inputs are required for the analyses.

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import seaborn as sns
import pickle
from scipy.spatial import cKDTree

project_dir = ""  # set top-level directory path
processed_data_dir = project_dir + "data/processed/limfac/"
colors = ['#2271B2', '#3DB7E9', '#F748A5', '#359B73', '#D55E00', '#E69F00', '#F0E442']

# open dataset
dsnb = xr.open_dataset(f"{processed_data_dir}nonborder_limfac_allAC_rmS_ERA5_GRIB_allcorr.nc")  # cell limiting factors
dsv = xr.open_dataset(f"{processed_data_dir}vert_limfac_allAC_rmS_ERA5_GRIB_allcorr_v3.nc")  # vertical ppcf region boundaries
dsh = xr.open_dataset(f"{processed_data_dir}limfac_allAC_rmS_ERA5_GRIB_allcorr_v3.nc")  # horizontal ppcf region boundaries

dsh

## Limiting factors per grid cell (non-border)

The goal of the first analysis is to compare the global distribution of the three limiting factors (persistence, droplet formation, droplet freezing). For each grid cell, we determine which of the three factors limits persistent contrail formation. Persistence and droplet freezing are independent of the aircraft design and thus only have one bar per factor. We also show the impact of the RHi enhancements ("corrections") using markers. 

The following notebook cell produces Figure 3 of the linked paper. To save the figure, uncomment the final line and choose a saving location.

In [ ]:
ds = dsnb

colors = ['#2271B2', '#3DB7E9', '#F748A5', '#359B73', '#D55E00', '#E69F00', '#F0E442']
plt_vars = ["ppcf", "frm", "frz", "per"]

# corrections
corr_labels = ["98% RHi", "95% RHi", "90% RHi"]
corr_lst = ds.corr.values
marker_styles = ['o', 's', '^', 'd', 'v']  # different marker styles for corrections

# choose aircraft to show and order
ac_order = ["AC8", "AC7", "AC3", "AC0", "AC1", "AC4"]
ac_labels = ["WET-75", "WET-50", "HYB-80", "CON-LG", "CON-NG", "H2C-04"]
n_ac = len(ac_order)

with open(processed_data_dir+"areas_grib.pickle", "rb") as f:
    areas = pickle.load(f)

# plot figure
fig, ax = plt.subplots()
ax2 = ax.twinx()
width = 1./(len(ds.AC.values)+1)

# plot ppcf 
for i_ac, ac_id in enumerate(ac_order):
    vals = (ds.sel(AC=ac_id, corr="uncor")["ppcf"].sum("level") * areas).sum("values") / sum(areas) / len(ds.level)
    # print("ppcf: ", ac_id, vals.data)
    ax.bar(i_ac*width, vals, width, color=colors[i_ac], label=ac_labels[i_ac], zorder=3)
    
    # plot corrections
    for i_corr, corr in enumerate(corr_lst[1:], start=1):
        corr_vals = (ds.sel(AC=ac_id, corr=corr)["ppcf"].sum("level") * areas).sum("values") / sum(areas) / len(ds.level)
        ax.scatter(i_ac*width, corr_vals, marker=marker_styles[i_corr-1], s=6, c="k", zorder=5)

# plot per
per_vals = 1 - ((ds.sel(corr="uncor")["per"].sum("level") * areas).sum("values") / sum(areas) / len(ds.level))
# print("Persistence: ", per_vals.data)
ax2.bar((n_ac+3)*width, per_vals, width, color="tab:gray", edgecolor="k", zorder=3)
for i_corr, corr in enumerate(corr_lst[1:], start=1):
    corr_vals = 1 - ((ds.sel(corr=corr)["per"].sum("level") * areas).sum("values") / sum(areas) / len(ds.level))
    ax2.scatter((n_ac+3)*width, corr_vals, marker=marker_styles[i_corr-1], s=6, c="k", zorder=5)

# plot frm
for i_ac, ac_id in enumerate(ac_order):
    vals = 1 - ((ds.sel(AC=ac_id, corr="uncor")["frm"].sum("level") * areas).sum("values") / sum(areas) / len(ds.level))
    # print("Formation: ", ac_id, vals.data)
    ax2.bar((n_ac+5+i_ac)*width, vals, width, color=colors[i_ac], zorder=3)
    
    # plot corrections
    for i_corr, corr in enumerate(corr_lst[1:], start=1):
        corr_vals = 1 - ((ds.sel(AC=ac_id, corr=corr)["frm"].sum("level") * areas).sum("values") / sum(areas) / len(ds.level))
        ax2.scatter((n_ac+5+i_ac)*width, corr_vals, marker=marker_styles[i_corr-1], s=6, c="k", zorder=5)

# plot frz (no corrections for freezing)
frz_vals = 1 - ((ds.sel(corr="uncor")["frz"].sum("level") * areas).sum("values") / sum(areas) / len(ds.level))
# print("Freezing: ", frz_vals.data)
ax2.bar((2*n_ac+6)*width, frz_vals, width, color="tab:gray", edgecolor="k", zorder=3)


# set axis ticks and labels
ax.set_ylabel("Global area- and level-weighted $p_{pcf}$ [-]")
ax.set_ylim([0, 0.18])
ax.set_yticks(np.arange(0, 0.20, 0.02))
ax2.set_ylabel("Normalised global area- and level-weighted\nlimiting factor [-]")
ax2.set_ylim([0, 1])
ax.set_xticks(width*np.array([1/2*(n_ac-1), n_ac+3, n_ac+5+1/2*(n_ac-1), 2*n_ac+6]))
ax.set_xticklabels(["Potential persistent\ncontrail formation", "Persistence", "Droplet\nformation", "Droplet\nfreezing"])

# create custom grid
ax.axvline(1, c="k", lw=1, zorder=1)
ypos_lst = [0.2, 0.4, 0.6, 0.8]
for ypos in ypos_lst:
    ax2.axhline(ypos, 0.403, 1, c="gray", lw=0.2, zorder=-1, alpha=0.7)

# create custom correction legend
from matplotlib.legend import Legend
corr_handles = [plt.Line2D([0], [0], marker=marker_styles[i], color='k', linestyle='None', markersize=4) for i in range(len(corr_labels))]
corr_legend = Legend(ax2, corr_handles, corr_labels, loc='upper right', title="Corrections", framealpha=1)

ax.legend(loc="upper left", title="Aircraft")
ax2.add_artist(corr_legend)
fig.tight_layout()
# fig.savefig("figs/nonborder_limfac.pdf")

We now analyse the latitude- and altitude-dependence of the three limiting factors. The following notebook cell creates Supplementary Figure 4 of the linked paper. To save the figure, uncomment the final line and choose a saving location.

In [ ]:
ds_frm = dsnb.sel(corr="uncor")["frm"].groupby("latitude").mean(dim="values")
ds_frz = dsnb.sel(corr="uncor")["frz"].groupby("latitude").mean(dim="values")
ds_per = dsnb.sel(corr="uncor")["per"].groupby("latitude").mean(dim="values")

ac_order = ["AC8", "AC7", "AC3", "AC0", "AC1", "AC4"]
ac_labels = ["WET-75", "WET-50", "HYB-80", "CON-LG", "CON-NG", "H2C-04"]

fig, axs = plt.subplots(7, figsize=(7, 7), sharex=True)

lvl_axes = [ax.twinx() for ax in axs]
for j, (ax, lvl_ax) in enumerate(zip(axs[::-1], lvl_axes[::-1])):
    ax.plot(ds_per.latitude.values, 1 - ds_per.isel(level=j).values, c="k", ls="--", zorder=3)
    ax.plot(ds_frz.latitude.values, 1 - ds_frz.isel(level=j).values, c="k", ls=":", zorder=3)
    for i, AC in enumerate(ac_order):
        ax.plot(ds_frm.latitude.values, 1 - ds_frm.isel(level=j).sel(AC=AC).values,
                c=colors[i], zorder=1, alpha=0.8)
        
    
    # create secondary y-axis to show level value
    lvl_ax.set_ylim([0, 1])
    lvl_ax.set_yticks([0.5])
    lvl_ax.set_yticklabels([f"{ds_frm.level.values[j]:.0f} hPa"])

for ax in axs:
    ax.set_xlim([-90, 90])
    ax.set_xticks([-90, -60, -30, 0, 30, 60, 90])
    ax.set_ylim([-0.1, 1.1])
    ax.set_yticks([0.0, 0.5, 1.0])
    ax.grid(visible=True, which='major', lw=0.1, c="gray", alpha=0.2)
    if ax == axs[0]:
        ax.set_yticklabels([0, "", 1])
    else:
        ax.set_yticklabels([0, "", 1])

# disable x axes to avoid clutter
for ax, lvl_ax in zip(axs, lvl_axes):
    if ax != axs[0]:
        ax.spines["top"].set_visible(False)
        lvl_ax.spines["top"].set_visible(False)
    if ax != axs[-1]:
        ax.spines["bottom"].set_visible(False)
        lvl_ax.spines["bottom"].set_visible(False)

# add legend
from matplotlib.lines import Line2D
custom_lines = [Line2D([0], [0], color="k", lw=2, ls="-"),
                Line2D([0], [0], color="k", lw=2, ls=":"),
                Line2D([0], [0], color="k", lw=2, ls="--")]
axs[-1].legend(custom_lines, ["Formation", "Freezing", "Persistence"], loc="center", ncol=1, handlelength=3, prop={'size': 8})

# add full figure labels
fig.text(0.04, 0.5, "Normalised area-weighted limiting factor [-]", va='center', rotation='vertical')
axs[-1].set_xlabel("Latitude [deg]")
plt.subplots_adjust(hspace=0, wspace=0.175)
# fig.savefig("figs/nonborder_limfac-lat.pdf")

Finally, we analyse the latitude-, longitude- and altitude-dependence of potential persistent contrail formation for each aircraft design considered. You can change the aircraft and level shown in the plot by changing `ac_ids` and `levels` respectively. The size of the figure `figsize` should also be changed accordingly. Note that for all aircraft and levels, this analysis can take a few minutes to run.

In [ ]:
# show global ppcf
ac_ids = ["AC8", "AC7", "AC3", "AC0", "AC1", "AC4"]
levels = [350.0, 300.0, 250.0, 225.0, 200.0, 175.0, 150.0]
ds = dsh.sel(AC=ac_ids, level=levels, corr="uncor")

# Define the target regular grid (1280x640)
target_lat = np.linspace(ds.latitude.values.min(), ds.latitude.values.max(), 640)
target_lon = np.linspace(ds.longitude.values.min(), ds.longitude.values.max(), 1280)
target_grid_lon, target_grid_lat = np.meshgrid(target_lon, target_lat)
target_points_flat = np.vstack((target_grid_lat.flatten(), target_grid_lon.flatten())).T

# Determine the number of levels and aircrafts
n_levels = len(levels)
n_ac = len(ac_ids)

# Set up the figure with a grid of subplots
fig, axes = plt.subplots(nrows=n_levels, ncols=n_ac, figsize=(23.5, 14),
                         subplot_kw={'projection': ccrs.PlateCarree()})

plt.subplots_adjust(wspace=0.03, hspace=0.03)
# Flatten axes for easy iteration
axes = axes.flatten()

# Flatten lat and lon values for KDTree
lat_flat = ds.latitude.values.flatten()
lon_flat = ds.longitude.values.flatten()
points_flat = np.vstack((lat_flat, lon_flat)).T
tree = cKDTree(points_flat)

# define vmin and vmax
vmin = 0.
vmax = 0.4

# Loop over levels and aircraft to plot each subplot
for i, level in enumerate(levels[::-1]):
    for j in range(n_ac):
        ax = axes[i * n_ac + j]
        
        # Flatten values for the current level and aircraft
        vals_flat = ds["ppcf"].sel(level=level).isel(AC=j).values.flatten()

        # Find nearest neighbors
        _, indices = tree.query(target_points_flat, k=1)

        # Use indices to gather values data
        target_vals_flat = vals_flat[indices]
        target_vals = target_vals_flat.reshape(640, 1280)

        # Convert the interpolated values back into an xarray Dataset
        target_ds = xr.Dataset({'vals': (('latitude', 'longitude'), target_vals)},
                               coords={'latitude': ('latitude', target_lat),
                                       'longitude': ('longitude', target_lon)})

        # Plot results
        im = target_ds.vals.plot(ax=ax, transform=ccrs.PlateCarree(), cmap="Oranges", add_colorbar=False, vmin=vmin, vmax=vmax)
        ax.coastlines()
        if i == 0:
            ax.set_title(f"{ac_ids[j]}")

# Add a single colorbar for all plots
fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.92, 0.12, 0.02, 0.75])
fig.colorbar(im, cax=cbar_ax, label="Potential persistent contrail formation $p_{PCF}$")

## Horizontal and vertical limiting factors (borders)

Next, we analyse the boundaries of potential persistent contrail formation regions in both horizontal and vertical direction. Our objective is to understand the relative importance of the limiting factors in defining the horizontal and vertical boundaries of persistent contrail formation regions. We do this by calculating the length of the boundaries caused by each limiting factor and comparing that to the total perimeter of all regions. Here, we create Figure 4 of the linked paper, which sums all values across all pressure levels and 10 years of ERA5 data. To save the figure, uncomment the final line and choose a saving location.

In [ ]:
fig, (ax1, ax2) = plt.subplots(ncols=2, figsize=(12, 5))

i_lvl = None  # index or None
plt_vars = ["limfac_tot", "limfac_per", "limfac_frm", "limfac_frz"]
var_labels = ["Total", "Persistence", "Droplet\nformation", "Droplet\nfreezing"]

# choose aircraft to show and order
ac_order = ["AC8", "AC7", "AC3", "AC0", "AC1", "AC4"]
ac_labels = ["WET-75", "WET-50", "HYB-80", "CON-LG", "CON-NG", "H2C-04"]
width = 1./(len(ac_order)+1)  # bar width

# corrections
corr_labels = ["98% RHi", "95% RHi", "90% RHi"]
corr_lst = dsv.corr.values
marker_styles = ['o', 's', '^', 'd', 'v']  # different marker styles for different datasets

# open perimeters and areas
with open(processed_data_dir+"perimeters_grib.pickle", "rb") as f:
    perimeters_dict = pickle.load(f)
perimeters = np.array(list(perimeters_dict.values()))  # convert to 1D array for normalisation
with open(processed_data_dir+"areas_grib.pickle", "rb") as f:
    areas = pickle.load(f)


ax11 = ax1.twinx()
ax21 = ax2.twinx()

# "Total" on outside axis, the rest compared to the total
# plot total on outside axis
for ds, ax, mult, fac10 in zip((dsh, dsv), (ax1, ax2), (perimeters, areas), (1e6, 1e8)):
    for i_ac, ac_id in enumerate(ac_order):
        if i_lvl == None:
            vals = (ds.isel(corr=0).sel(AC=ac_id)["limfac_tot"].sum("level") * mult).sum("values")
        else:
            vals = (ds.isel(corr=0, level=i_lvl).sel(AC=ac_id)["limfac_tot"] * mult).sum("values")
        ax.bar(i_ac*width, vals / fac10, width, label=ac_labels[i_ac], color=colors[i_ac], zorder=3)

        for i_corr, corr in enumerate(corr_lst[1:], start=1):
            if i_lvl == None:
                scatter_vals = (ds.isel(corr=i_corr).sel(AC=ac_id)["limfac_tot"].sum("level") * mult).sum("values")
            else:
                scatter_vals = (ds.isel(corr=i_corr, level=i_lvl).sel(AC=ac_id)["limfac_tot"] * mult).sum("values")
            ax.scatter(i_ac*width, scatter_vals / fac10, marker=marker_styles[i_corr - 1], s=6, c="k", zorder=5)
        

# plot comparative results
for ds, ax, mult in zip((dsh, dsv), (ax11, ax21), (perimeters, areas)):
    # plot bars for main dataset
    for i_var, plt_var in enumerate(plt_vars[1:], start=1):
        for i_ac, ac_id in enumerate(ac_order):
            if i_lvl == None:
                vals = (ds.isel(corr=0).sel(AC=ac_id)[plt_var].sum("level") * mult).sum("values")
                vals_total = (ds.isel(corr=0).sel(AC=ac_id)["limfac_tot"].sum("level") * mult).sum("values")
            else:
                vals = (ds.isel(corr=0, level=i_lvl).sel(AC=ac_id)[plt_var] * mult).sum("values")
                vals_total = (ds.isel(corr=0, level=i_lvl).sel(AC=ac_id)["limfac_tot"] * mult).sum("values")
            ax.bar(0.2+i_var+i_ac*width, vals / vals_total * 100., width, color=colors[i_ac], zorder=3)

    # plot markers for other datasets
    for i_corr, corr in enumerate(corr_lst[1:], start=1):
        for i_var, plt_var in enumerate(plt_vars[1:], start=1):
            for i_ac, ac_id in enumerate(ac_order):
                if i_lvl is None:
                    vals = (ds.isel(corr=i_corr).sel(AC=ac_id)[plt_var].sum("level") * mult).sum("values")
                    vals_total = (ds.isel(corr=i_corr).sel(AC=ac_id)["limfac_tot"].sum("level") * mult).sum("values")
                else:
                    vals = (ds.isel(corr=i_corr, level=i_lvl).sel(AC=ac_id)[plt_var] * mult).sum("values")
                ax.scatter(0.2+i_var+i_ac*width, vals / vals_total * 100., marker=marker_styles[i_corr - 1], s=6, c="k", zorder=5)
                
    # set ticks
    ax.set_xticks(np.arange(len(plt_vars))+width/2.*(len(dsv.AC.values)-1)+[0, 0.2, 0.2, 0.2], plt_vars)
    ax.set_xticklabels(var_labels)



# set boundary lines
ax1.axvline(0.95, c="k", lw=1, zorder=1)
ax2.axvline(0.95, c="k", lw=1, zorder=1)

# create custom grid for 0 -> 100 %
ypos_lst = [20, 40, 60, 80, 100]
for ypos in ypos_lst:
    ax11.axhline(ypos, 0.276, 1, c="gray", lw=0.2, zorder=-1, alpha=0.7)
    ax21.axhline(ypos, 0.276, 1, c="gray", lw=0.2, zorder=-1, alpha=0.7)
    

# axis settings
ax1.set_ylabel("Horizontal limfac total perimeter [10$^6$ km]")
ax2.set_ylabel("Vertical limfac total area [10$^8$ km$^2$]")

# set y limits to be approximately equal on CON-LG
ax1.set_ylim([0, 8])
ax2.set_ylim([0, 3.7])
ax11.set_ylim([0, 140])
ax21.set_ylim([0, 140])
ax11.set_yticks([0, 20, 40, 60, 80, 100])
ax11.set_yticklabels([None, "20 %", "40 %", "60 %", "80 %", "100 % of total"], ha="right", va="bottom")
ax11.tick_params(axis="y", direction="in", pad=-9)
ax21.set_yticks([0, 20, 40, 60, 80, 100])
ax21.set_yticklabels([None, None, "40 %", "60 %", "80 %", "100 % of total"], ha="right", va="bottom")
ax21.tick_params(axis="y", direction="in", pad=-9)

# add subplot labels
fig.text(0.065, 0.90, "a", fontsize=18)
fig.text(0.575, 0.90, "b", fontsize=18)

# create custom legend
from matplotlib.legend import Legend
handles, labels = ax2.get_legend_handles_labels()
corr_handles = [plt.Line2D([0], [0], marker=marker_styles[i], color='k', linestyle='None', markersize=4) for i in range(len(corr_labels))]
corr_legend = Legend(ax1, corr_handles, corr_labels, loc='upper left', bbox_to_anchor=(0.76, 1), title="Corrections")
ac_legend = Legend(ax2, handles[:len(ac_labels)], ac_labels, loc='upper left', ncol=2, bbox_to_anchor=(0.53, 1), title="Aircraft")
ax1.add_artist(corr_legend)
ax2.add_artist(ac_legend)

fig.tight_layout()
plt.subplots_adjust(wspace=0.18)
# fig.savefig("figs/limfac_tot_handv_perc.pdf")

Next, we wish to analyse the latitude- and level-dependence of each limiting factor and the potential persistent contrail formation. In the following notebook cell, you can define `limfac_str` as limfac_frm, limfac_frz, limfac_per or ppcf. You can also decide whether you want to see the horizontal limiting factors (use `dsh`) or vertical limiting factors (use `dsv`).

In [ ]:
limfac_str = "ppcf"  # choice of limfac_frm, limfac_frz, limfac_per, ppcf

ds_latg = dsh[limfac_str].sel(corr="uncor").groupby("latitude").mean(dim="values")

ac_order = ["AC8", "AC7", "AC3", "AC0", "AC1", "AC4"]
ac_labels = ["WET-75", "WET-50", "HYB-80", "CON-LG", "CON-NG", "H2C-04"]

fig, axs = plt.subplots(7, figsize=(7, 7), sharex=True, sharey=True)
plt.subplots_adjust(wspace=0, hspace=0)
lvl_axes = [ax.twinx() for ax in axs]
for j, (ax, lvl_ax) in enumerate(zip(axs[::-1], lvl_axes[::-1])):
    for i, AC in enumerate(ac_order):
        ax.plot(ds_latg.latitude.values, ds_latg.isel(level=j).sel(AC=AC).values, c=colors[i], label=ac_labels[i])
    
    # create secondary y-axis to show level value
    lvl_ax.set_ylim([0, 1])
    lvl_ax.set_yticks([0.5])
    lvl_ax.set_yticklabels([f"{ds_latg.level.values[j]:.0f} hPa"])

for ax in axs:
    ax.grid(visible=True, which='major', lw=0.1, c="gray")

# add legend
axs[-1].legend(loc="upper center", ncol=3, prop={'size': 8})

# add full figure labels
fig.text(0.04, 0.5, limfac_str, va='center', rotation='vertical')
axs[-1].set_xlabel("Latitude [deg]")

The previous figure does not consider where aircraft actually fly, it only shows a "potential". Therefore, we have used the DEPA 2050 progressive global aviation scenario in the year 2050 to weight the potential persistent contrail formation (Figure 5 of the linked paper). We see that the tropical increase in persistent contrail formation at 225 to 250 hPa for hydrogen-powered aircraft is now less pronounced because there are less aircraft flying in this region. Instead, the most important areas for persistent contrail formation are the northern extratropics between 300 and 250 hPa, which are the lower cruise altitudes for typical airliners. 

To save the figure, uncomment the final line and choose a saving location.

In [ ]:
# loading data
ds = dsh
depa_dir = project_dir + "data/external/DEPA2050/"
df_depa = pd.read_csv(f"{depa_dir}FP_con_2050.txt", delim_whitespace=True, header=None,
                      names=["longitude", "latitude", "level", "fuel", "NOx", "dist", "temp"])

# choose aircraft to show and order
ac_lst = ["AC8", "AC7", "AC3", "AC0", "AC1", "AC4"]
ac_labels = ["WET-75", "WET-50", "HYB-80", "CON-LG", "CON-NG", "H2C-04"]

# processing for Figure 1
i_corr = 0
ds_latg = ds.isel(corr=i_corr)["ppcf"].groupby("latitude").mean(dim="values")

# processing for Figure 2
levels = [350, 300, 250, 225, 200, 175, 150]
level_bins = [375, 325, 275, 237.5, 212.5, 187.5, 162.5, 137.5]
lat_bins = np.arange(-90.5, 91.5, 1)
lat_bin_labels = np.arange(-90, 91, 1)

# processing DEPA 2050 data
df_depa_red = df_depa.drop(columns=["NOx", "fuel", "temp"])
df_depa_red['level_bin'] = pd.cut(df_depa_red['level'], bins=level_bins[::-1], labels=levels[::-1])
df_depa_red = df_depa_red.dropna(subset=['level_bin'])
df_depa_grp = df_depa_red.groupby(['latitude', 'level_bin'], as_index=False)['dist'].sum()
multi_index = pd.MultiIndex.from_product([lat_bin_labels, levels[::-1]], names=["latitude", "level_bin"])
df_depa_grp = df_depa_grp.set_index(["latitude", "level_bin"]).reindex(multi_index, fill_value=0).reset_index()
df_depa_grp.rename({"level_bin": "level"}, axis=1, inplace=True)
ds_depa_grp = df_depa_grp.set_index(["latitude", "level"]).to_xarray()['dist']

# processing ds to fit 1° resolution
ds_latg_red = ds_latg.groupby_bins('latitude', bins=lat_bins, labels=lat_bin_labels).mean()
ds_latg_red = ds_latg_red.rename({'latitude_bins': 'latitude'})

# calculating resulting fuel-weighted ds_res
ds_res = ds_latg_red * ds_depa_grp / ds_depa_grp.max()  # weighting by maximum value

# Create combined figure
fig, axs = plt.subplots(7, 2, figsize=(14, 7), sharex=True)

# Create secondary y-axes for the left subplot
lvl_axes_left = [ax.twinx() for ax in axs[:, 0]]

# Plot Figure 1 on the left side
for j, (ax, lvl_ax) in enumerate(zip(axs[::-1, 0], lvl_axes_left[::-1])):
    for i, ac_id in enumerate(ac_lst):
        ax.plot(ds_latg.latitude.values, ds_latg.isel(level=j).sel(AC=ac_id).values, c=colors[i], label=ac_labels[i])
        ax.set_xlim([-90, 90])
        ax.set_xticks([-90, -60, -30, 0, 30, 60, 90])
        ax.grid(visible=True, which='major', lw=0.1, c="gray", alpha=0.2)

    # draw secondary y-axis for levels
    lvl_ax.set_ylim([0, 1])
    lvl_ax.set_yticks([0.5])
    lvl_ax.set_yticklabels([f"{ds_latg.level.values[j]:.0f} hPa"], fontsize=14)

# Plot Figure 2 on the right side
for j, ax in enumerate(axs[::-1, 1]):
    ax_left = ax.twinx()
    for i, ac_id in enumerate(ac_lst):
        vals = ds_res.isel(level=j).sel(AC=ac_id).values
        ax.plot(ds_res.latitude.values, vals, c=colors[i], label=ac_labels[i])
        ax.grid(visible=True, which='major', lw=0.1, c="gray", alpha=0.2)

    ax.yaxis.set_ticks_position('right')
    ax.yaxis.set_label_position('right')
    
    ax.set_ylim([-0.008, 0.12])
    if j == 6:
        ax.set_yticks([0.0, 0.04, 0.08, 0.12])
    else:
        ax.set_yticks([0.0, 0.04, 0.08])
        
    # Hide secondary y-axis for the right plot
    ax_left.set_ylim([0., 1.])
    ax_left.set_yticks([0.5])
    ax_left.set_yticklabels([""])
    ax_left.yaxis.set_ticks_position('left')

# Formatting both plots
for ax_col in axs:
    ax_col[0].set_ylim([-0.02, 0.3])
    
    if ax_col[0] == axs[0, 0]:
        ax_col[0].set_yticks([0.0, 0.1, 0.2, 0.3])
    else:
        ax_col[0].set_yticks([0.0, 0.1, 0.2])

# Add legends and labels
handles, labels = axs[-1, 0].get_legend_handles_labels()
axs[0, 1].legend(handles, labels, loc="upper center", ncol=3, prop={'size': 12}, framealpha=1)

# add subplot labels
fig.text(0.06, 0.925, "a", fontsize=18)
fig.text(0.54, 0.925, "b", fontsize=18)

# add labels
axs[3, 0].set_ylabel("Potential persistent contrail formation $p_{PCF}$ [-]", fontsize=14)
axs[3, 1].set_ylabel("$p_{PCF}$ weighted by distance flown [-]", fontsize=14)
axs[6, 0].set_xlabel("Latitude [deg]", fontsize=14)
axs[6, 1].set_xlabel("Latitude [deg]", fontsize=14)

fig.tight_layout()
plt.subplots_adjust(hspace=0, wspace=0.175)
# fig.savefig("figs/lat-vs-ppcf_weighted.pdf")

We now look at the effect of the RHi enhancement (Supplementary Figure 5).

In [ ]:
ds_latg = dsh.sel(AC="AC4")["ppcf"].groupby("latitude").mean(dim="values")

fig, axs = plt.subplots(7, figsize=(7, 7), sharex=True)
plt.subplots_adjust(wspace=0, hspace=0)
lvl_axes = [ax.twinx() for ax in axs]
for j, (ax, lvl_ax) in enumerate(zip(axs[::-1], lvl_axes[::-1])):
    for i, cor in enumerate(ds.corr.values):
        ax.plot(ds_latg.latitude.values, ds_latg.isel(level=j, corr=i).values, c=colors[i], label=cor)
    
    # create secondary y-axis to show level value
    lvl_ax.set_ylim([0, 1])
    lvl_ax.set_yticks([0.5])
    lvl_ax.set_yticklabels([f"{ds_latg.level.values[j]:.0f} hPa"])

for ax in axs:
    ax.set_ylim([-0.02, 0.4])
    ax.grid(visible=True, which='major', lw=0.1, c="gray", alpha=0.5)
    if ax == axs[0]:
        ax.set_yticks([0.0, 0.2, 0.4])
    else:
        ax.set_yticks([0.0, 0.2])

# add legend
axs[-1].legend(loc="upper center", ncol=2, prop={'size': 8})

# add full figure labels
fig.text(0.04, 0.5, "Potential persistent contrail formation $p_{PCF}$", va='center', rotation='vertical')
axs[-1].set_xlabel("Latitude [deg]")
# fig.savefig("figs/ppcf_RHicorr.pdf")

## Mean over latitude bands

To give us some numerical idea of the increases, we calculate the mean over each latitude band (southern extratropics, tropics, northern extratropics). In the plot below, we show the $p_{pcf}$ and distance-weighted $p_{pcf}$ for CON-LG (last generation conventional kerosene aircraft) as dotted black lines. We then calculate the mean values across each latitude band for all aircraft and plot them using the same colours as before. The numerical results shown in Table 2 and Supplementary Table 1 can be obtained from `ds_comp` (next cell).

In [ ]:
fig, axs = plt.subplots(7, 2, figsize=(14, 7), sharex=True)

# Plot the left side
for j, ax in enumerate(axs[::-1, 0]):
    ax.plot(ds_latg.latitude.values, ds_latg.isel(level=j).sel(AC="AC0").values, "k:")
    
    for i, ac_id in enumerate(ac_lst):
        # plot mean values
        ax.axhline(ds_latg.sel(AC=ac_id, latitude=slice(-90, -30)).isel(level=j).mean(), xmax=1/3, c=colors[i])
        ax.axhline(ds_latg.sel(AC=ac_id, latitude=slice(-30, 30)).isel(level=j).mean(), xmin=1/3, xmax=2/3, c=colors[i])
        ax.axhline(ds_latg.sel(AC=ac_id, latitude=slice(30, 90)).isel(level=j).mean(), xmin=2/3, xmax=1, c=colors[i])
    
    ax.axvline(-30, c="gray", ls="--", lw=1)
    ax.axvline(30, c="gray", ls="--", lw=1)
    ax.set_xlim([-90, 90])
    ax.set_ylim([-0.02, 0.3])

# Plot the right side
for j, ax in enumerate(axs[::-1, 1]):
    ax.plot(ds_res.latitude.values, ds_res.isel(level=j).sel(AC="AC0").values, "k:")
    
    for i, ac_id in enumerate(ac_lst):
        # plot mean values
        ax.axhline(ds_res.sel(AC=ac_id, latitude=slice(-90, -30)).isel(level=j).mean(), xmax=1/3, c=colors[i])
        ax.axhline(ds_res.sel(AC=ac_id, latitude=slice(-30, 30)).isel(level=j).mean(), xmin=1/3, xmax=2/3, c=colors[i])
        ax.axhline(ds_res.sel(AC=ac_id, latitude=slice(30, 90)).isel(level=j).mean(), xmin=2/3, xmax=1, c=colors[i])
    
    ax.axvline(-30, c="gray", ls="--", lw=1)
    ax.axvline(30, c="gray", ls="--", lw=1)
    ax.set_xlim([-90, 90])
    ax.set_ylim([-0.008, 0.12])

fig.tight_layout()
plt.subplots_adjust(hspace=0, wspace=0.175)

In [ ]:
# ppcf changes compared to reference
ac_lst = ["AC8", "AC7", "AC3", "AC0", "AC1", "AC4"]
ac_labels = ["WET-75", "WET-50", "HYB-80", "CON-LG", "CON-NG", "H2C-04"]
lvl_lst = [350.0, 300.0, 250.0, 225.0, 200., 175.0, 150.0, slice(350, 150)]
lvl_labels = ["350", "300", "250", "225", "200", "175", "150", "all"]
lat_lst = [slice(-90, -30), slice(-30, 30), slice(30, 90), slice(-90, 90)]
lat_labels = ["xtropS", "trop", "xtropN", "all"]

ppcf_arr = np.empty((len(ac_lst), len(lvl_lst), len(lat_lst)))
ppcf_arr_w = np.empty((len(ac_lst), len(lvl_lst), len(lat_lst)))
comp_arr = np.empty((len(ac_lst), len(lvl_lst), len(lat_lst)))
comp_arr_w = np.empty((len(ac_lst), len(lvl_lst), len(lat_lst)))

for i_ac, ac in enumerate(ac_lst):
    for i_lvl, lvl in enumerate(lvl_lst):
        ds_latg_i = ds_latg.sel(level=lvl)
        ds_res_i = ds_res.sel(level=lvl)
        if lvl_labels[i_lvl] == "all":
            ds_latg_i = ds_latg_i.mean("level")
            ds_res_i = ds_res_i.mean("level")
        for i_lat, lat in enumerate(lat_lst):
            ds_i_mean = ds_latg_i.sel(AC=ac, latitude=lat).mean("latitude")
            ds_AC0_mean = ds_latg_i.sel(AC="AC0", latitude=lat).mean("latitude")
            ppcf_arr[i_ac, i_lvl, i_lat] = ds_i_mean.values
            comp_arr[i_ac, i_lvl, i_lat] = (ds_i_mean / ds_AC0_mean).values * 100.
            
            ds_res_i_mean = ds_res_i.sel(AC=ac, latitude=lat).mean("latitude")
            ds_res_AC0_mean = ds_res_i.sel(AC="AC0", latitude=lat).mean("latitude")
            ppcf_arr_w[i_ac, i_lvl, i_lat] = ds_res_i_mean.values
            comp_arr_w[i_ac, i_lvl, i_lat] = (ds_res_i_mean / ds_res_AC0_mean).values * 100.
            

ds_comp = xr.Dataset({"ppcf_comp": (["AC", "level", "lat_band"], comp_arr),
                      "ppcf_comp_weighted": (["AC", "level", "lat_band"], comp_arr_w),
                      "ppcf": (["AC", "level", "lat_band"], ppcf_arr),
                      "ppcf_weighted": (["AC", "level", "lat_band"], ppcf_arr_w)},
                     coords = {"AC": ac_lst,
                               "level": lvl_labels,
                               "lat_band": lat_labels})

ds_comp